<h3><b>Análise de dados ContosoRetailDW: Desvendando insights de negócio com SQL.</h3></b>

<b>Introdução:</b>

Como analista de dados, a capacidade de extrair informações de grandes conjuntos de dados é fundamental. Este projeto tem como objetivo principal a pratica e aplicação do SQL para extrair, manipular e analisar dados para obter insights a partir do banco de dados ContosoRetailDW.

<b>Objetivo:</b>

O principal objetivo deste projeto é aprimorar e exibir minhas proficiências em SQL por meio de uma análise abrangente dos dados da Contoso. Minha meta é responder a perguntas de negócio cruciais, realizar análises aprofundadas e validar os resultados, culminando em conclusões acionáveis e recomendações estratégicas para impulsionar o sucesso da Contoso. Este exercício servirá como um portfólio prático de minhas capacidades analíticas e de manipulação de dados em SQL.

In [2]:
import pyodbc
import pandas as pd
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore', category=UserWarning,)


load_dotenv()

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"  
    f"SERVER={os.getenv('DB_SERVER')};"
    "DATABASE=ContosoRetailDW;"  
    f"UID={os.getenv('DB_USER')};"
    f"PWD={os.getenv('DB_PASSWORD')}"
)


<h4><b>1. Qual o total de vendas (SalesAmount) e quantidade de vendas (SalesQuantity) por ano e mês?</h4></b>

In [3]:
query = """
SELECT
    DD.CalendarYear AS Year,
    DD.CalendarMonth AS MonthNumber,
    DD.CalendarMonthLabel AS MonthName,
    SUM(FS.SalesAmount) AS TotalSalesAmount,
    SUM(FS.SalesQuantity) AS TotalSalesQuantity,
    SUM(FS.SalesAmount) / NULLIF(SUM(FS.SalesQuantity), 0) AS AverageUnitPrice
FROM
    FactSales FS WITH (NOLOCK)
JOIN
    DimDate DD WITH (NOLOCK) ON FS.DateKey = DD.Datekey
GROUP BY
    DD.CalendarYear,
    DD.CalendarMonth,
    DD.CalendarMonthLabel
ORDER BY
    DD.CalendarYear,
    DD.CalendarMonth;
"""

TotalSales = pd.read_sql(query, conn)

TotalSales.head()


,Year,MonthNumber,MonthName,TotalSalesAmount,TotalSalesQuantity,AverageUnitPrice
0,2007,200701,January,2.698353e+08,1164359,231.7457
1,2007,200702,February,2.982160e+08,1160226,257.0326
2,2007,200703,March,3.004869e+08,1158003,259.4871
3,2007,200704,April,4.001603e+08,1540164,259.8167
4,2007,200705,May,4.234291e+08,1578798,268.1971


In [4]:
df_annual_summary = TotalSales.groupby('Year').agg(
    TotalSalesAmount=('TotalSalesAmount', 'sum'),
    TotalSalesQuantity=('TotalSalesQuantity', 'sum')
).reset_index()

df_annual_summary['AverageUnitPrice'] = df_annual_summary['TotalSalesAmount'] / df_annual_summary['TotalSalesQuantity']

df_annual_summary['AnnualGrowth_Amount'] = df_annual_summary['TotalSalesAmount'].pct_change() * 100
df_annual_summary['AnnualGrowth_Quantity'] = df_annual_summary['TotalSalesQuantity'].pct_change() * 100

df_annual_summary

,Year,TotalSalesAmount,TotalSalesQuantity,AverageUnitPrice,AnnualGrowth_Amount,AnnualGrowth_Quantity
0,2007,4.561941e+09,16984310,268.597368,NaN,NaN
1,2008,4.111234e+09,16511811,248.987439,-9.879729,-2.781973
2,2009,3.740483e+09,19824333,188.681411,-9.017985,20.061531


<h4><b>2. Quais são os 10 produtos mais vendidos em termos de SalesAmount e SalesQuantity no último ano disponível?</h4></b>

In [7]:
query = """
SELECT TOP 10
    DP.ProductName AS [Product], 
    SUM(FS.SalesAmount) AS [TotalSalesAmount],
    SUM(FS.SalesQuantity) AS [TotalSalesQuantity]
FROM 
    FactSales AS FS 
INNER JOIN 
    DimProduct AS DP ON FS.ProductKey = DP.ProductKey
INNER JOIN 
    DimDate AS DD ON FS.DateKey = DD.Datekey
INNER JOIN (
    SELECT MAX(CalendarYear) AS LastYear 
    FROM DimDate AS DD2
    WHERE EXISTS (
        SELECT 1 FROM FactSales AS FS2 
        WHERE FS2.DateKey = DD2.Datekey
    )
) AS LY ON DD.CalendarYear = LY.LastYear
GROUP BY 
    DP.ProductName
ORDER BY 
    [TotalSalesAmount] DESC, 
    [TotalSalesQuantity] DESC;
"""
TopSales = pd.read_sql(query, conn)

TopSales

,Product,TotalSalesAmount,TotalSalesQuantity
0,Fabrikam Laptop19 M9000 Black,1.323961e+07,12221
1,Fabrikam Laptop19W M9800 Black,1.320351e+07,11183
2,Adventure Works Laptop19W X1980 White,1.186201e+07,9308
3,Adventure Works Laptop19W X1980 Silver,1.140962e+07,8935
4,Litware Refrigerator 24.7CuFt X980 Brown,1.140022e+07,3631
5,Adventure Works Laptop19W X1980 Blue,1.134571e+07,8904
6,Litware Refrigerator 24.7CuFt X980 Grey,1.107129e+07,3527
7,Fabrikam Laptop16W M6080 Black,1.056897e+07,11096
8,Litware Refrigerator 24.7CuFt X980 Blue,1.056864e+07,3370
9,Adventure Works Laptop19W X1980 Red,1.046928e+07,8198


<h4><b>3. Qual o desempenho de vendas (SalesAmount e TotalCost) por canal de vendas (Loja Física, Online, Catálogo, Revendedor)?</h4></b>

In [6]:
query = """ 
SELECT
    DC.ChannelName,
    SUM(FS.SalesAmount) AS TotalSalesAmount,
    SUM(FS.TotalCost) AS TotalCost
FROM
    FactSales AS FS
JOIN
    DimChannel AS DC ON FS.ChannelKey = DC.ChannelKey
GROUP BY
    DC.ChannelName
ORDER BY
    TotalSalesAmount DESC;
"""
SalesByChanel = pd.read_sql(query, conn)

SalesByChanel

,ChannelName,TotalSalesAmount,TotalCost
0,Store,6.942853e+09,2.992580e+09
1,Online,2.677599e+09,1.163294e+09
2,Reseller,1.715198e+09,7.435589e+08
3,Catalog,1.078008e+09,4.654637e+08


<h4><b>4. Qual a receita total (SalesAmount) e o custo total (TotalCost) para cada loja no último ano disponível com dados de vendas?</h4></b>

In [8]:
query = """ 
SELECT
    DS.StoreName AS [Store],
    SUM(FS.SalesAmount) AS [TotalRevenue],
    SUM(FS.TotalCost) AS [TotalCost],
    SUM(FS.SalesAmount) - SUM(FS.TotalCost) AS [Profit],
    COUNT(*) AS [Transactions],
    SUM(FS.SalesAmount) / NULLIF(COUNT(*), 0) AS [AverageTicket]
FROM 
    FactSales FS WITH (NOLOCK)
INNER JOIN 
    DimStore DS WITH (NOLOCK) ON FS.StoreKey = DS.StoreKey
INNER JOIN 
    DimDate DD WITH (NOLOCK) ON FS.DateKey = DD.Datekey
INNER JOIN (
    SELECT TOP 1 CalendarYear AS LastYear
    FROM DimDate DD2 WITH (NOLOCK)
    WHERE EXISTS (
        SELECT 1 FROM FactSales FS2 WITH (NOLOCK)
        WHERE FS2.DateKey = DD2.Datekey
    )
    ORDER BY CalendarYear DESC
) AS LY ON DD.CalendarYear = LY.LastYear
GROUP BY 
    DS.StoreName
ORDER BY 
    [TotalRevenue] DESC;
"""

SalesByStore = pd.read_sql(query, conn)

SalesByStore

,Store,TotalRevenue,TotalCost,Profit,Transactions,AverageTicket
0,Contoso North America Online Store,3.394581e+08,1.478951e+08,1.915630e+08,72377,4690.1374
1,Contoso Asia Online Store,3.393642e+08,1.487604e+08,1.906038e+08,66797,5080.5306
2,Contoso Catalog Store,3.143032e+08,1.360144e+08,1.782888e+08,58871,5338.8466
3,Contoso Europe Online Store,2.723278e+08,1.178783e+08,1.544495e+08,58700,4639.3154
4,Contoso North America Reseller,1.939603e+08,8.399320e+07,1.099671e+08,41562,4666.7713
...,...,...,...,...,...,...
301,Contoso Roma Store,4.042906e+06,1.726953e+06,2.315953e+06,1182,3420.3945
302,Contoso Berlin Store,3.743782e+06,1.620604e+06,2.123178e+06,1136,3295.5827
303,Contoso Marseille Store,3.645385e+06,1.565798e+06,2.079586e+06,1129,3228.8614
304,Contoso Racine No.2 Store,2.485142e+06,1.090944e+06,1.394198e+06,670,3709.1676


Clientes

<h4><b>5. Qual a distribuição de clientes por gênero e estado/província (StateProvinceName)?</h4></b>

In [9]:
query = """ 
SELECT
    DC.Gender,
    DG.StateProvinceName,
    COUNT(DC.CustomerKey) AS NumberOfCustomers
FROM
    DimCustomer AS DC
JOIN
    DimGeography AS DG ON DC.GeographyKey = DG.GeographyKey
GROUP BY
    DC.Gender,
    DG.StateProvinceName
ORDER BY
    DC.Gender,
    NumberOfCustomers DESC;
"""

CustumerDemographicsByLocation = pd.read_sql(query, conn)

CustumerDemographicsByLocation

,Gender,StateProvinceName,NumberOfCustomers
0,None,California,45
1,None,Washington,34
2,None,Texas,32
3,None,Massachusetts,19
4,None,Wisconsin,18
...,...,...,...
175,M,Arizona,1
176,M,Texas,1
177,M,Kentucky,1
178,M,Minnesota,1


In [ ]:
query = """ 
SELECT TOP 5
    FOS.CustomerKey,
    DC.FirstName,  
    DC.MiddleName, 
    DC.LastName, 
    SUM(FOS.SalesAmount) AS [TotalSales],
    SUM(FOS.SalesQuantity) AS [TotalItens]
FROM
    FactOnlineSales FOS WITH (NOLOCK)
INNER JOIN
    DimCustomer DC WITH (NOLOCK) ON FOS.CustomerKey = DC.CustomerKey
WHERE
    YEAR(FOS.DateKey) = (
        SELECT MAX(YEAR(DateKey))
        FROM FactOnlineSales WITH (NOLOCK)
    )
GROUP BY
    FOS.CustomerKey,
    DC.FirstName,
    DC.MiddleName,
    DC.LastName
ORDER BY
    [TotalSales] DESC;
"""

TopCustomers = pd.read_sql(query, conn)

TopCustomers

,CustomerKey,FirstName,MiddleName,LastName,TotalSales,TotalItens
0,19069,None,None,None,1.492568e+07,65824
1,19068,None,None,None,1.382731e+07,65815
2,19071,None,None,None,1.362224e+07,65826
3,19072,None,None,None,1.328849e+07,65774
4,19066,None,None,None,1.309057e+07,65834


Com um objetivo claro e direto, este projeto foi dedicado à prática aprofundada da linguagem SQL. A construção e execução de diversas queries permitiram-me solidificar o entendimento sobre a manipulação de bancos de dados, extração de informações relevantes e a lógica por trás de consultas eficientes.